In [1]:
import pandas as pd
# import dummy classifier
from sklearn.dummy import DummyClassifier
# import train_test_split
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,66728,21770,1,156,64.0,140,80,2,1,0,0,1,1
1,69098,21876,1,170,85.0,160,90,1,1,0,0,1,1
2,59185,23270,1,151,90.0,130,80,1,1,0,0,1,1
3,49288,19741,1,159,97.0,120,80,1,1,0,0,1,1
4,62481,18395,1,164,68.0,120,80,1,1,0,0,1,0


In [4]:
x_train, x_val, y_train, y_val = train_test_split(data.drop('cardio', axis=1), data['cardio'], test_size=0.2, random_state=0)

In [5]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)

dummy_clf.score(x_val, y_val)

0.4988392857142857

In [6]:
#logistic regression algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)

lr_clf.score(x_val, y_val)

0.6957142857142857

In [15]:
#import min max scaler (normalizar os dados), one hot encoder (transformar em binários os dados categóricos), and column transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# pode deixar dessa forma, se tiver mt ruim a gente muda
var_to_be_used = ['gluc', 'cholesterol', 'ap_hi', 'age', 'ap_lo', 'weight']
numerical_features = ['ap_hi', 'ap_lo', 'age', 'weight']
categorical_features = ['gluc', 'cholesterol']

# create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train = preprocessor.fit_transform(x_train)
x_val = preprocessor.transform(x_val)

In [16]:
# import ransearchcv
from sklearn.model_selection import RandomizedSearchCV

# define the parameters
params = {
    'solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'tol' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'C' : [1.0, 2.0, 3.0, 4.0],
    'penalty' : ['l1', 'l2', 'elasticnet'],
    'dual' : [0, 1],
}
# create the model
lr_clf = LogisticRegression()

# create the random search cv
random_search = RandomizedSearchCV(lr_clf, param_distributions = params, n_iter = 5, scoring = 'roc_auc', n_jobs = -1, cv = 5, verbose = 3)

# fit the model
random_search.fit(x_train, y_train)

random_search.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/home/camila/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/camila/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/camila/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/camila/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/camila/.local/lib/python3.10/site-packages/sklearn

[CV 2/5] END C=4.0, dual=0, penalty=elasticnet, solver=newton-cholesky, tol=0.0001;, score=nan total time=   0.0s
[CV 4/5] END C=4.0, dual=0, penalty=elasticnet, solver=newton-cholesky, tol=0.0001;, score=nan total time=   0.0s
[CV 3/5] END C=2.0, dual=1, penalty=elasticnet, solver=liblinear, tol=0.001;, score=nan total time=   0.0s
[CV 5/5] END C=4.0, dual=0, penalty=elasticnet, solver=newton-cholesky, tol=0.0001;, score=nan total time=   0.0s
[CV 3/5] END C=4.0, dual=0, penalty=elasticnet, solver=newton-cholesky, tol=0.0001;, score=nan total time=   0.0s
[CV 1/5] END C=4.0, dual=0, penalty=elasticnet, solver=newton-cholesky, tol=0.0001;, score=nan total time=   0.0s
[CV 1/5] END C=2.0, dual=1, penalty=elasticnet, solver=liblinear, tol=0.001;, score=nan total time=   0.0s
[CV 2/5] END C=2.0, dual=1, penalty=elasticnet, solver=liblinear, tol=0.001;, score=nan total time=   0.0s
[CV 4/5] END C=2.0, dual=1, penalty=elasticnet, solver=liblinear, tol=0.001;, score=nan total time=   0.0s
[C

/home/camila/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/camila/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/camila/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/camila/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    

{'tol': 0.001, 'solver': 'sag', 'penalty': 'l2', 'dual': 0, 'C': 4.0}

In [17]:
# train lr with the best params
lr_clf = LogisticRegression(**random_search.best_params_)
lr_clf.fit(x_train, y_train)

/home/camila/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


LogisticRegression(C=4.0, dual=0, solver='sag', tol=0.001)

In [18]:
lr_clf.score(x_val, y_val)

0.6469642857142858

In [19]:
data_test = pd.read_csv('test.csv')

In [20]:
data_test['cardio'].shape

(14000,)

In [21]:
x_test = preprocessor.transform(data_test)
y_test = lr_clf.predict(x_test)

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(data_test['cardio'], y_test)

0.6500714285714285